# Eliminacion Recursiva de Features (RFE)

Es una tecnica utilizada para seleccionar las caracteristicas mas importantes de un conjunto de datos. Su objetivo es eliminar las caracteristicas menos relevantes, mejorando asi el rendimiento del modelo y reduciendo su complejidad.

Generalmente, RFE se aplica a modelos que pueden proporcionar alguna medida de la importancia de las caracteristicas como los modelos lineales o arboles de decision. Entre los modelos donde se puede aplicar tenemos:

- Regresion Lineal
- Regresion Logistica
- Arboles de decision
- Bosques aleatorios
- Maquina de vectores de soporte
- Entre otros...

## Como funciona RFE?

   1. **Entrenamiento inicial**: Entrena un modelo utilizando todas las caracteristicas (Variables predictoras)
    
    
   2. **Evaluacion de Importancia**: Evalua la importancia de cada caracteristica. Esta importancia se puede medir de diferentes maneras dependiendo del modelo utilizado (coeficientes de la regresion lineal)
   
    
   3. **Eliminacion de Caracteristicas**: Elimina las caracteristicas menos importantes para el modelo
   
   
   4. **Repeticion de pasos**: Repite los pasos de 1 a 3 hasta alcanzar un numero predefinido de caracteristicas.

## Aplicacion de RFE

Supongamos que tenemos un conjunto de 5 caracteristicas y vamos a usar RFE para seleccionar las 3 caracteristicas mas importantes. Generamos el conjunto de datos de prueba convenientemente.

In [1]:
# Generamos datos de prueba
import numpy as np
import pandas as pd

# semilla
np.random.seed(0)

# 5 caracteristicas
X = pd.DataFrame({
    'feature_1': np.random.rand(100),
    'feature_2': np.random.rand(100),
    'feature_3': np.random.rand(100),
    'feature_4': np.random.rand(100),
    'feature_5': np.random.rand(100)
})

# variable objetivo
# la asociamos linealmente con 3 de las 5 caracteristicas
y = 3*X['feature_2'] + 2*X['feature_5'] + X['feature_1'] + np.random.rand(100)

Al asociar la variable objetivo con 3 de las 5 features nos aseguramos que el modelo solo va a estar influenciado por estas tres caracteristicas lo cual debe verse reflejado al aplicar RFE.

In [2]:
# juntamos los datos en un dataframe para mejorar la visualizacion de los datos
df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(X.shape[1])])
df['target'] = y
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,target
0,0.548814,0.677817,0.311796,0.906555,0.401260,3.695163
1,0.715189,0.270008,0.696343,0.774047,0.929291,3.756831
2,0.602763,0.735194,0.377752,0.333145,0.099615,3.532546
3,0.544883,0.962189,0.179604,0.081101,0.945302,6.072647
4,0.423655,0.248753,0.024679,0.407241,0.869489,3.242399


### 1.- Entrenamiento inicial
Entrenamos el modelo de regresion lineal con todas las caracteristicas y calculamos la importancia de cada una.

In [3]:
# libreria para usar la regresion lineal
from sklearn.linear_model import LinearRegression

# funcion para entrena el modelo
def train_model(X, y):
    model = LinearRegression()
    return model.fit(X,y)

In [4]:
# entrenamos el modelo
modelo = train_model(X, y)

### 2.- Evaluacion de importancia
Para el caso de la regresion, debemos obtener los coeficientes de cada caracteristicas una vez entrenado el modelo.

In [5]:
# funcion para obtener los coeficientes de cada caracteristica
def evaluate_model(model):
    coeficientes = model.coef_
    return coeficientes

In [6]:
# coeficiente de las caracteristicas
coefi = evaluate_model(modelo)
coefi

array([1.0169788 , 2.96313024, 0.01281724, 0.04742558, 1.94336239])

### 3.- Eliminacion de caracteristicas
Identificamos la caracteristica con el coeficiente mas pequeño (el valor absoluto) y la eliminamos

In [7]:
# funcion que determina la caracteristica con menor valor de coeficiente y la elimina
def remove_feature(X, coeficientes):
    # indice del coficiente mas pequeño
    min_indice = np.argmin(np.abs(coeficientes))
    # nombre de la caracteristica correspondiente al indice anterior
    feature_remove = X.columns[min_indice]
    # eliminar caracteristica
    X = X.drop(columns=[feature_remove])
    # retornar matriz de caracteristica y nombre de la caracteristica eliminada
    return X, feature_remove
    

### 4.- Repetir hasta obtener el numero deseado de caracteristicas

Inicialmente dijimos que vamos a obtener las 3 caracteristicas mas importantes. Entonces, repetimos el proceso del 1 al 3 hasta obtener las tres caracteristicas. Para automatizar el proceso, procedemos como sigue

In [8]:
# numero de features a seleccionar
n_features = 3

# ejecutar proceso hasta llegar a 3 caracteristicas
while X.shape[1] > n_features:
    modelo = train_model(X,y)
    coefi = evaluate_model(modelo)
    X, feature_remove = remove_feature(X, coefi)
    
# mostrar las 3 caracteristicas mas importantes
print('Caracteristicas seleccionadas', X.columns)
    
    

Caracteristicas seleccionadas Index(['feature_1', 'feature_2', 'feature_5'], dtype='object')


Los resultados confirman que los features 1, 2 y 5 son los que realmente influyen sobre el modelo de regresion lineal.

## Aplicacion de RFE con Scikit-Learn

Ahora vamos a utilizar la libreria Scikit Learn para aplicar este metodo y simplificar el proceso de eliminacion recursiva. Trabajaremos sobre los mismos datos de prueba del ejemplo anterior para comparar los resultados 

In [14]:
# features
X = df.drop(columns=['target'])
# variable objetivo
y = df['target']

Aplicamos RFE con la libreria sklearn

In [15]:
# libreria para aplicar RFE
from sklearn.feature_selection import RFE

# crear modelo de regresion
modelo_2 = LinearRegression()

# Crear el objeto RFE y especificar el numero de caracteristicas deseadas
rfe = RFE(estimator=modelo_2, n_features_to_select=3)

# ajustar RFE a los datos
rfe.fit(X, y)

# obtener las caracteristicas seleccionadas por RFE
features_select = X.columns[rfe.support_]

# mostrar nombre
features_select

Index(['feature_1', 'feature_2', 'feature_5'], dtype='object')

De esta forma, obtenemos los features mas importantes para el modelo de regresion, que en este caso son los features 1, 2 y 5, coincidiendo con los resultados del ejemplo anterior.

# Eliminacion Recursiva de Features con Validacion Cruzada (RFECV)

Es una tecnica de seleccion de caracteristicas que mejora la metodologia RFE al incoporar la validacion cruzada para seleccionar automaticamente el numero optimo de caracteristicas

## Como funciona RFECV?

1. **Entrenamiento del modelo**: Similar a RFE, entrena el modelo usando todas las caracteristicas


2. **Validacion Cruzada**: Utiliza la validacion cruzada para evaluar el rendimiento del modelo en diferentes subconjuntos de datos.


3. **Eliminacion de caracteristicas**: Elimina la caracteristica menos importante y evalua el modelo usando validacion cruzada.


4. **Repeticion de pasos**: Repite los pasos 1 hasta 3 hasta que se encuentre el conjunto optimo de caracteristicas que maximiza el rendimiento del modelo.

## Aplicacion de RFECV

Para implementar este metodos, vamos a utilizar los datos de Boston Housing de una libreria de Scikit Learn. Cargamos los datos

In [19]:
from sklearn.datasets import load_boston

# cargamos los datos
boston = load_boston()

# features
X = pd.DataFrame(boston.data, columns=boston.feature_names)
# variable objetivo
y = pd.Series(boston.target)

/home/jbernal/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing 

In [20]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [21]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
dtype: float64